### additional_analysis_line_graph_of_performance
- 最終更新: 2024/2/15, 15:50

- additional_analysisで行った、採点基準表に対して

#### Google Driveのマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#ライブラリのインポート
import numpy as np
import pandas as pd
import json

In [3]:
#classification_reportを辞書型で格納する関数の宣言
import re

first_row_pattern = "\s+"

def classification_report_to_dict(input_classification_report):
  """
  classification_reportを辞書型で格納する
  """
  classification_report_dict = dict()
  for idx, line in enumerate(input_classification_report.split("\n")):
    #評価値キーの準備
    if idx == 0:
      line = line.lstrip(" ")
      line = re.sub(first_row_pattern, " ", line)
      keys = line.split(" ")

      for tmp_key in keys:
        classification_report_dict[tmp_key] = dict()

    #評価値の取得
    if idx in [2, 3, 4]:
      line = line.lstrip(" ")
      line = re.sub(first_row_pattern, " ", line)
      keys = line.split(" ")

      #格納していく
      classification_report_dict["precision"][keys[0]] = keys[1]
      classification_report_dict["recall"][keys[0]] = keys[2]
      classification_report_dict["f1-score"][keys[0]] = keys[3]
      classification_report_dict["support"][keys[0]] = keys[4]

    elif idx == 6:
      line = line.lstrip(" ")
      line = re.sub(first_row_pattern, " ", line)
      keys = line.split(" ")

      #格納していく
      classification_report_dict["f1-score"][keys[0]] = keys[1]
      classification_report_dict["support"][keys[0]] = keys[2]

    elif idx in [7, 8]:
      line = line.lstrip(" ")
      line = re.sub(first_row_pattern, " ", line)
      line = re.sub("macro avg", "macro_avg", line)
      line = re.sub("weighted avg", "weighted_avg", line)
      keys = line.split(" ")

      #格納していく
      classification_report_dict["precision"][keys[0]] = keys[1]
      classification_report_dict["recall"][keys[0]] = keys[2]
      classification_report_dict["f1-score"][keys[0]] = keys[3]
      classification_report_dict["support"][keys[0]] = keys[4]

  return classification_report_dict

In [4]:
#テスト: 単発
classification_report_dir = "/content/drive/MyDrive/Colaboratory/Work/sst_auto_scoring/result/additional_analysis/combination_split_answer/Fold0/classification_report.txt"
with open(classification_report_dir, "r") as input_file:
  classification_report = input_file.read()

print(classification_report)
print(classification_report_to_dict(classification_report))

              precision    recall  f1-score   support

           2       0.17      0.30      0.21       120
           1       0.73      0.59      0.65       630
           0       0.73      0.76      0.75       559

    accuracy                           0.64      1309
   macro avg       0.54      0.55      0.54      1309
weighted avg       0.68      0.64      0.65      1309

{'precision': {'2': '0.17', '1': '0.73', '0': '0.73', 'macro_avg': '0.54', 'weighted_avg': '0.68'}, 'recall': {'2': '0.30', '1': '0.59', '0': '0.76', 'macro_avg': '0.55', 'weighted_avg': '0.64'}, 'f1-score': {'2': '0.21', '1': '0.65', '0': '0.75', 'accuracy': '0.64', 'macro_avg': '0.54', 'weighted_avg': '0.65'}, 'support': {'2': '120', '1': '630', '0': '559', 'accuracy': '1309', 'macro_avg': '1309', 'weighted_avg': '1309'}}


In [5]:
tmp_classification_df =  pd.DataFrame.from_dict(classification_report_to_dict(classification_report))
tmp_classification_df

,precision,recall,f1-score,support
2,0.17,0.30,0.21,120
1,0.73,0.59,0.65,630
0,0.73,0.76,0.75,559
macro_avg,0.54,0.55,0.54,1309
weighted_avg,0.68,0.64,0.65,1309
accuracy,NaN,NaN,0.64,1309


In [6]:
tmp_classification_df_rev = tmp_classification_df.copy()
tmp_classification_df_rev["exp"] = "Fold0"
tmp_classification_df_rev

,precision,recall,f1-score,support,exp
2,0.17,0.30,0.21,120,Fold0
1,0.73,0.59,0.65,630,Fold0
0,0.73,0.76,0.75,559,Fold0
macro_avg,0.54,0.55,0.54,1309,Fold0
weighted_avg,0.68,0.64,0.65,1309,Fold0
accuracy,NaN,NaN,0.64,1309,Fold0


In [28]:
#全体を処理して、研究1と研究2の結果をまとめる
#各Foldの平均をとる

#研究1
rubric_part_answer_df = pd.DataFrame()

RESULT_DIR = "/content/drive/MyDrive/Colaboratory/Work/sst_auto_scoring/result/additional_analysis/combination_split_answer/"

for num_fold in range(10):
  num_fold_key = "Fold" + str(num_fold)

  #classification_reportの読み込み
  result_dir = RESULT_DIR + num_fold_key + "/classification_report.txt"
  with open(result_dir, "r") as input_file:
    classification_report = input_file.read()
  classification_report_dict = classification_report_to_dict(classification_report)

  #データフレームにする
  classification_report_df = pd.DataFrame.from_dict(classification_report_dict)
  classification_report_df["Fold"] = num_fold_key

  rubric_part_answer_df = pd.concat([rubric_part_answer_df, classification_report_df])

rubric_part_answer_df

,precision,recall,f1-score,support,Fold
2,0.17,0.30,0.21,120,Fold0
1,0.73,0.59,0.65,630,Fold0
0,0.73,0.76,0.75,559,Fold0
macro_avg,0.54,0.55,0.54,1309,Fold0
weighted_avg,0.68,0.64,0.65,1309,Fold0
accuracy,NaN,NaN,0.64,1309,Fold0
2,0.20,0.29,0.24,120,Fold1
1,0.74,0.65,0.69,630,Fold1
0,0.76,0.79,0.77,559,Fold1
macro_avg,0.57,0.58,0.57,1309,Fold1


In [29]:
rubric_part_answer_df.reset_index(inplace=True)
rubric_part_answer_df = rubric_part_answer_df.rename({'index': 'measure'}, axis='columns')

In [30]:
rubric_part_answer_df

,measure,precision,recall,f1-score,support,Fold
0,2,0.17,0.30,0.21,120,Fold0
1,1,0.73,0.59,0.65,630,Fold0
2,0,0.73,0.76,0.75,559,Fold0
3,macro_avg,0.54,0.55,0.54,1309,Fold0
4,weighted_avg,0.68,0.64,0.65,1309,Fold0
5,accuracy,NaN,NaN,0.64,1309,Fold0
6,2,0.20,0.29,0.24,120,Fold1
7,1,0.74,0.65,0.69,630,Fold1
8,0,0.76,0.79,0.77,559,Fold1
9,macro_avg,0.57,0.58,0.57,1309,Fold1


In [31]:
#データの保存
rubric_part_answer_df.to_csv("/content/drive/MyDrive/Colaboratory/Work/sst_auto_scoring/result/additional_analysis/rubric_part_answer.csv", index=False)